---
# Scopes and Namespaces
---

- When we assign a value or an object to a variable (a = 10, means we created this label 'a' that points at some memory address and contains value = 10)
    - A portion of code where that name or biniding is defined is called the `Lexical scope` of the variable.
    - These bindings are stored somewhere, that is called `Namespaces`,*We can consider namesapces as a table, that contains the label and the reference that is pointing to*
    
---
## Global Scope
---

**Global scope is essentially the `module` scope, which spans a single file only**

- There is no concept of a truly global scope (across all the modules in our entire app) in Python
- The only exception to this is a `built-in` globally available objects, such as True,False, None etc...


# Closure

In [1]:
# Standard closure
def outer():
    x = 'python'
    def inner():
        print(x)
    return inner

In [3]:
fn = outer()

In [4]:
# To check the free variables
fn.__code__.co_freevars

('x',)

In [5]:
fn.__closure__

# This tells the free variable is pointing the cell object and cell object points to the string object at some location

(<cell at 0x060E4F30: str object at 0x03E67800>,)

In [6]:
def outer():
    count = 0
    def inner():
        nonlocal count
        count += 1
        return count
    
    return inner


In [7]:
fn = outer()

In [9]:
fn.__code__.co_freevars

('count',)

In [10]:
fn.__closure__

(<cell at 0x060DE390: int object at 0x5532D8A0>,)

In [11]:
hex(id(0))

'0x5532d8a0'

In [12]:
fn()

1

In [13]:
def pow(n):
    def inner(x):
        return x ** n
    return inner

In [15]:
fn = pow(5)

In [16]:
fn.__code__.co_freevars

('n',)

In [18]:
fn.__closure__

(<cell at 0x0634DFF0: int object at 0x5532D8F0>,)

In [19]:
def create_adders():
    adders = []
    for i in range(1,4):
        adders.append(lambda x : x+i)
    return adders

In [20]:
adders = create_adders()

In [21]:
adders

[<function __main__.create_adders.<locals>.<lambda>(x)>,
 <function __main__.create_adders.<locals>.<lambda>(x)>,
 <function __main__.create_adders.<locals>.<lambda>(x)>]

In [23]:
adders[0].__closure__

(<cell at 0x06864710: int object at 0x5532D8D0>,)

In [25]:
adders[1].__closure__

(<cell at 0x06864710: int object at 0x5532D8D0>,)

In [26]:
adders[1](10)

13

In [27]:
adders[0](10)

13

In [28]:
adders[2](10)

13

<strong> To chnage this, you can add defaut value to the lambda function</strong>

In [29]:
def create_adders():
    adders = []
    for i in range(1,4):
        # We can add default value because default values are created at the creation time
        # Here we are not even creating closure it is a normal function
        adders.append(lambda x, y=i: x + y)
    return adders

In [30]:
adders_ = create_adders()

In [32]:
adders_[0].__closure__

In [33]:
adders_[0](10)

11

In [34]:
adders_[1](10)

12

In [35]:
adders_[2](10)

13

<strong> This means we can also pass second value instead of default </strong>

<strong> For example</strong>

In [36]:
adders_[0](10, 11)

21

In [37]:
adders_[1](10,11)

21

--------------------------------
# Applications of closure
-----------------------------------

<strong style="color:#1FDA9C">Class vs Closure</strong>

**What you can write as a class can be written as closure instead**



In [41]:
# Using class
class Averager:
    def __init__(self):
        self.numbers = []
    
    def add(self, number):
        self.numbers.append(number)
        total = sum(self.numbers)
        count = len(self.numbers)
        return total/count

In [42]:
a = Averager()

In [43]:
a.add(10)

10.0

In [44]:
a.add(20)

15.0

In [45]:
a.add(30)

20.0

**We can do the same thing with closure**

In [47]:
def averager():
    numbers = []
    def add(number):
        # Here numbers is a free variabel
        numbers.append(number)
        total = sum(numbers)
        count = len(numbers)
        return total/count
        
    return add
    

In [49]:
a = averager()

<span style="color:red"> Counter </span>
<hr>
Here we want to be able to provide an initial value and this closure will increment that value by 1

In [52]:
def counter(initial_value=0):
    def inc(increment=1):
        # We cannot assign something from outer scope 
        # Here we either have to mention global or nonlocal
        nonlocal initial_value
        initial_value += 1
        return initial_value
    return inc

In [53]:
counter1 = counter()

In [55]:
counter1()

1

In [56]:
counter1()

2

In [57]:
counter1()

3

**Suppose we want to keep track of, how many times the program has ran**

In [58]:
def counter(fn):
    '''
    Here, fn is the function which we want to keep track of 
    '''
    cnt = 0
    def inner(*args, **kwargs):
        '''
        This function will take arbitrarty amount of positional and keyword only argument
        '''
        nonlocal cnt
        cnt += 1
        print('{0} has been called {1} times'.format(fn.__name__, cnt))
        return fn(*args, **kwargs)
    return inner

In [59]:
def add(a, b):
    return a+b

In [60]:
def mult(a, b):
    return a*b

In [64]:
counter_add = counter(add)

In [68]:
result1 = counter_add(3,5)
result1

add has been called 4 times


8

In [69]:
counter_mult = counter(mult)

In [71]:
result2 = counter_mult(3,5)
result2

mult has been called 1 times


15

**Another way we can achieve this is by using `dictionary`**

In [99]:
counters = {}
def counter(fn):
    cnt = 0
    def inner(*args,**kwargs):
        nonlocal cnt
        cnt += 1
        counters[fn.__name__] = cnt
        return fn(*args, **kwargs)
    return inner

In [100]:
counted_add = counter(add)
counted_mult = counter(mult)

In [101]:
counted_add(10,20)

30

In [102]:
counted_add(10,50)

60

In [103]:
counters

{'add': 2}

In [104]:
counted_mult(10,20)

200

In [105]:
counters

{'add': 2, 'mult': 1}

**Suppose if I do not want to remember that I have to remember to define global variable dictionary first**

In [106]:
def counter(fn, counters):
    cnt = 0
    def inner(*args,**kwargs):
        nonlocal cnt
        cnt += 1
        counters[fn.__name__] = cnt
        return fn(*args, **kwargs)
    return inner

In [107]:
new_counters = {}
counted_add = counter(add, new_counters)
counted_mult = counter(mult, new_counters)

In [108]:
counted_add(2,3)

5

In [109]:
counted_mult(5,6)

30

In [110]:
counters

{'add': 2, 'mult': 1}

In [111]:
new_counters

{'add': 1, 'mult': 1}

In [112]:
# The new cpunter dictionary has been updated 

**Factorial function**

In [113]:
def fact(n):
    prod = 1
    for i in range(2,n+1):
        prod *= i
    return prod

In [114]:
fact(3)

6

In [115]:
fact(6)

720

In [120]:
counted_fact = counter(fact, new_counters)

In [122]:
counted_fact(10)

3628800

In [123]:
new_counters

{'add': 1, 'mult': 1, 'fact': 1}

**We do not need to rename it to counted_fact or counted_add or whatever counted, we can assign it to the same name**

**For example**

In [124]:
fact = counter(fact, new_counters)

In [125]:
# fact just works as before
fact(5)

120

In [126]:
new_counters

{'add': 1, 'mult': 1, 'fact': 1}

In [127]:
# Because fact is no more a funtion it has become `closure`

In [133]:
fact.__closure__
# int which is count
# dictionaty which is new_counter
# function which is fucntion which we pass in, in this case fact

(<cell at 0x0BE1B250: int object at 0x5532D8B0>,
 <cell at 0x0BE1B8B0: dict object at 0x0BE726F0>,
 <cell at 0x0BE1B750: function object at 0x0BDE9B70>)

In [134]:
fact.__code__.co_freevars

('cnt', 'counters', 'fn')

----------
# Decorators
----------
<ol>
    <li> Decorator function takes a function as an argument </li>
    <li> It returns a closure </li>
    <li> The closure usually accepts any combination of parameters using *args and **kwargs </li>
    <li> Run some code in the inner function (closure) </li>
    <li> It then calls original function (the function passed as an argument)</li>
    <li> It will run that function get the result back from that and returns that result back </li>

</ol>


# Decorator Application (Timing)

In [1]:
def timed(fn):
    '''
    We are using import at function level because if we use this function elsewhere,
    this import will be available with it
    '''
    from time import perf_counter
    from functools import wraps
    
    @wraps(fn)
    def inner(*args, **kwargs):
        '''
        This function will time how long the function fn takes to run
        '''
        start = perf_counter()
        result = fn(*args, **kwargs)
        end = perf_counter()
        elapsed = end - start
        

In [7]:
from functools import reduce

def add(a,b):
    return a+b

initial = (1,0)

dummy = range(5)

reduce(lambda prev, n: (prev[0]+prev[1], prev[0]),dummy,initial)

(8, 5)

In [3]:
1,2,3,5,8,13,21,34,55,89

88